In [1]:
# The code was removed by Watson Studio for sharing.

[Row(game_size='37', party_size='2', player_assists='0', player_dist_ride='2870.724', player_dist_walk='1784.84778', player_dmg='117', player_kills='1', player_survive_time='1106.32', team_placement='18'),
 Row(game_size='37', party_size='2', player_assists='0', player_dist_ride='2938.4072300000003', player_dist_walk='1756.0797100000002', player_dmg='127', player_kills='1', player_survive_time='1106.315', team_placement='18'),
 Row(game_size='37', party_size='2', player_assists='0', player_dist_ride='0.0', player_dist_walk='224.157562', player_dmg='67', player_kills='0', player_survive_time='235.558', team_placement='33'),
 Row(game_size='37', party_size='2', player_assists='0', player_dist_ride='0.0', player_dist_walk='92.93515', player_dmg='0', player_kills='0', player_survive_time='197.553', team_placement='33'),
 Row(game_size='37', party_size='2', player_assists='0', player_dist_ride='2619.07739', player_dist_walk='2510.447', player_dmg='175', player_kills='2', player_survive_time

In [2]:
#convert to integer & double

from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

df_data_1=df_data_1.withColumn("game_size", df_data_1["game_size"].cast(IntegerType()))
df_data_1=df_data_1.withColumn("party_size", df_data_1["party_size"].cast(IntegerType()))
df_data_1=df_data_1.withColumn("player_assists", df_data_1["player_assists"].cast(IntegerType()))
df_data_1=df_data_1.withColumn("player_dist_ride", df_data_1["player_dist_ride"].cast(DoubleType()))
df_data_1=df_data_1.withColumn("player_dist_walk", df_data_1["player_dist_walk"].cast(DoubleType()))
df_data_1=df_data_1.withColumn("player_dmg", df_data_1["player_dmg"].cast(IntegerType()))
df_data_1=df_data_1.withColumn("player_kills", df_data_1["player_kills"].cast(IntegerType()))
df_data_1=df_data_1.withColumn("player_survive_time", df_data_1["player_survive_time"].cast(DoubleType()))
df_data_1=df_data_1.withColumn("team_placement", df_data_1["team_placement"].cast(IntegerType()))


In [3]:
#print schema to check data type

df_data_1.cache()
df_data_1.printSchema()

root
 |-- game_size: integer (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_placement: integer (nullable = true)



In [4]:
#statistik deskriptif

df_data_1.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
game_size,13849287,47.14748044430013,25.176718925400998,4,100
party_size,13849287,2.7334386239522654,1.2463910518229826,1,4
player_assists,13849287,0.22962662265573672,0.5757951131748313,0,13
player_dist_ride,13849287,1156.4243585450847,1995.3956191797552,0.0,474871.2
player_dist_walk,13849287,1282.1640659438353,4109.221049409863,0.0,1188375.0
player_dmg,13849287,123.91849457665221,169.60369997787683,0,6343
player_kills,13849287,0.8871920987701388,1.5553867623402131,0,64
player_survive_time,13849287,83509.84393521953,7.256157446323657E7,0.0,6.3648215E10
team_placement,13849287,24.46128901798338,20.37429054520374,0,100


In [5]:
#find data correlation

import six
for i in df_data_1.columns:
    if not( isinstance(df_data_1.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to team_placement for ", i, df_data_1.stat.corr('team_placement',i))

Correlation to team_placement for  game_size 0.6376611714922066
Correlation to team_placement for  party_size -0.582088856931328
Correlation to team_placement for  player_assists -0.24289626112917562
Correlation to team_placement for  player_dist_ride -0.3844235687257262
Correlation to team_placement for  player_dist_walk -0.17134437476548225
Correlation to team_placement for  player_dmg -0.30645810352029407
Correlation to team_placement for  player_kills -0.2711514958458521
Correlation to team_placement for  player_survive_time 0.0027264442862474977
Correlation to team_placement for  team_placement 1.0


In [6]:
#mapping features and team placemenet

from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['game_size', 'party_size', 'player_assists', 'player_dist_ride', 'player_dist_walk', 'player_dmg','player_kills','player_survive_time'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(df_data_1)
vhouse_df = vhouse_df.select(['features', 'team_placement'])
vhouse_df.show(3)

+--------------------+--------------+
|            features|team_placement|
+--------------------+--------------+
|[37.0,2.0,0.0,287...|            18|
|[37.0,2.0,0.0,293...|            18|
|[37.0,2.0,0.0,0.0...|            33|
+--------------------+--------------+
only showing top 3 rows



In [7]:
#split training to test

splits = vhouse_df.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

In [8]:
#linear regression
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='team_placement', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.477797733026,-0.1052954394,-0.776438348699,-0.00271654003314,-0.000263480282797,-0.013263953915,-1.31232522557,0.0]
Intercept: 8.686175638486466


In [9]:
#root mean and r square

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 13.317451
r2: 0.572752


In [10]:
#describe data training

train_df.describe().show()

+-------+------------------+
|summary|    team_placement|
+-------+------------------+
|  count|          11081229|
|   mean|24.458734586208806|
| stddev|  20.3742311487861|
|    min|                 0|
|    max|               100|
+-------+------------------+



In [12]:
#akurasi prediksi test data

lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","team_placement","features").show(10)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="team_placement",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+--------------+--------------------+
|        prediction|team_placement|            features|
+------------------+--------------+--------------------+
|21.822487522959726|            30|(8,[0,1,2,7],[30....|
| 15.43154605777642|            27|(8,[0,1,3,7],[27....|
|21.536246309130068|            19|(8,[0,1,3,7],[28....|
|19.315027679494786|            10|(8,[0,1,3,7],[28....|
|27.511325878718303|            42|(8,[0,1,3,7],[48....|
|30.921810314050447|            46|(8,[0,1,3,7],[49....|
| 29.02711027139467|            43|(8,[0,1,3,7],[49....|
|28.841092258761947|            45|(8,[0,1,3,7],[49....|
| 28.00150101026248|            40|(8,[0,1,3,7],[49....|
| 47.88334957340983|            78|(8,[0,1,3,7],[88....|
+------------------+--------------+--------------------+
only showing top 10 rows

R Squared (R2) on test data = 0.570619


In [14]:
#find RMSE on the test data set

test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 13.3508


In [15]:
#Decision Tree

from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'team_placement')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="team_placement", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 5.60863


In [16]:
#mana yang paling berpengaruh

dt_model.featureImportances

SparseVector(8, {0: 0.4431, 4: 0.2858, 7: 0.2712})

In [20]:
df_data_1.take(1)

#yang paling berpengaruh ternyata game size, wajar karena semakin kecil sisanya akan diisi bot

[Row(game_size=37, party_size=2, player_assists=0, player_dist_ride=2870.724, player_dist_walk=1784.84778, player_dmg=117, player_kills=1, player_survive_time=1106.32, team_placement=18)]

In [25]:
#gradient-boosted tree regression

from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'team_placement', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'team_placement', 'features').show(20)

+------------------+--------------+--------------------+
|        prediction|team_placement|            features|
+------------------+--------------+--------------------+
|22.247426346445415|            30|(8,[0,1,2,7],[30....|
| 21.78599994987354|            27|(8,[0,1,3,7],[27....|
| 22.49980542950105|            19|(8,[0,1,3,7],[28....|
| 20.68701152062016|            10|(8,[0,1,3,7],[28....|
|40.888593236998105|            42|(8,[0,1,3,7],[48....|
|41.828317942456785|            46|(8,[0,1,3,7],[49....|
| 41.17433503001345|            43|(8,[0,1,3,7],[49....|
|40.305009010751434|            45|(8,[0,1,3,7],[49....|
| 33.11011178610538|            40|(8,[0,1,3,7],[49....|
|  81.6196767933093|            78|(8,[0,1,3,7],[88....|
| 82.87934953789231|            83|(8,[0,1,3,7],[90....|
| 84.10118977322247|            76|(8,[0,1,3,7],[91....|
| 84.10118977322247|            75|(8,[0,1,3,7],[93....|
| 78.21996882128681|            85|(8,[0,1,3,7],[94....|
| 90.74570362256765|           

In [24]:
#Evaluate GBT

gbt_evaluator = RegressionEvaluator(
    labelCol="team_placement", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 5.33359
